In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.datasets import imdb
import numpy as np
np.random.seed(10)

In [22]:
import re 
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('',text)

In [23]:
import os
def read_files(filetype):
    path='data/aclImdb/'
    file_list=[]
    positive_path =path+filetype+'/pos/'
    for f in os.listdir(positive_path):
        file_list +=[positive_path+f]
    
    negative_path=path+filetype+'/neg/'
    for f in os.listdir(negative_path):
        file_list +=[negative_path+f]
    print('read',filetype,'files:',len(file_list))
    
    all_labels=([1]*12500+[0]*12500)
    all_texts=[]
    for fi in file_list:
        with open(fi,encoding='utf8') as file_input:
            all_texts +=[rm_tags("".join(file_input.readlines()))]
    return all_labels,all_texts

In [24]:
y_train,train_text =read_files("train")
y_test,test_text=read_files("test")
#建立token
token = Tokenizer(num_words=3800)
token.fit_on_texts(train_text)

read train files: 25000
read test files: 25000


In [25]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)

In [26]:
x_train =sequence.pad_sequences(x_train_seq,maxlen=380)
x_test = sequence.pad_sequences(x_test_seq,maxlen=380)

In [29]:
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

model = Sequential()
model.add(Embedding(output_dim=32,
                   input_dim=3800,
                   input_length=380))
model.add(Dropout(0.2))

model.add(LSTM(32))

model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=1,activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 380, 32)           121600    
_________________________________________________________________
dropout_5 (Dropout)          (None, 380, 32)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_5 (Dense)              (None, 256)               8448      
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 138,625
Trainable params: 138,625
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',metrics=['accuracy'])

In [31]:
train_history =model.fit(x_train,y_train,batch_size=100,
                        epochs=10,verbose=2,validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 34s - loss: 0.5007 - acc: 0.7446 - val_loss: 0.3836 - val_acc: 0.8450
Epoch 2/10
 - 32s - loss: 0.2729 - acc: 0.8901 - val_loss: 0.5015 - val_acc: 0.7772
Epoch 3/10
 - 30s - loss: 0.2287 - acc: 0.9114 - val_loss: 0.4400 - val_acc: 0.8082
Epoch 4/10
 - 30s - loss: 0.2019 - acc: 0.9240 - val_loss: 0.4013 - val_acc: 0.8282
Epoch 5/10
 - 30s - loss: 0.1724 - acc: 0.9356 - val_loss: 0.4505 - val_acc: 0.8220
Epoch 6/10
 - 29s - loss: 0.1661 - acc: 0.9357 - val_loss: 0.4137 - val_acc: 0.8436
Epoch 7/10
 - 30s - loss: 0.1395 - acc: 0.9486 - val_loss: 0.6633 - val_acc: 0.7914
Epoch 8/10
 - 29s - loss: 0.1209 - acc: 0.9556 - val_loss: 0.5301 - val_acc: 0.8336
Epoch 9/10
 - 30s - loss: 0.1158 - acc: 0.9577 - val_loss: 0.8034 - val_acc: 0.7614
Epoch 10/10
 - 30s - loss: 0.1036 - acc: 0.9622 - val_loss: 0.6650 - val_acc: 0.8130
